# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
display(data.head())
data.shape

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
s1 = data.groupby(['CustomerID', 'ProductName']).agg('sum')
s1

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
s2 = pd.pivot_table(s1, index='CustomerID', values='Quantity', columns='ProductName').fillna(0)
s2

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [40]:
distances = 1/1+squareform(pdist(s2, 'euclidean'))
distances = pd.DataFrame(distances, index=s2.index, columns=s2.index)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,12.916375,11.488088,12.224972,12.401754,12.090537,13.409674,12.045361,12.269428,12.489125,...,207.871941,214.180675,226.656819,199.232187,231.913404,221.501701,218.188858,229.628520,240.000000,230.773802
200,12.916375,1.000000,12.747340,13.083046,13.569805,13.288206,13.165525,13.083046,12.874342,13.000000,...,207.310446,213.635839,225.697575,198.139544,231.952376,221.202180,216.728997,229.010965,240.037654,230.704158
264,11.488088,12.747340,1.000000,12.489125,12.224972,12.445523,13.000000,12.401754,12.180340,12.747340,...,207.387984,213.946003,226.435135,198.600607,231.371439,220.136943,217.612557,229.081126,239.266657,230.773802
356,12.224972,13.083046,12.489125,1.000000,13.083046,12.789826,13.328828,12.135529,12.958261,13.165525,...,207.649462,214.082144,226.452878,198.494304,232.038958,220.952268,218.437347,229.098663,239.493186,230.464594
412,12.401754,13.569805,12.224972,13.083046,1.000000,12.704700,13.328828,12.135529,12.789826,12.747340,...,207.900942,212.679002,226.572605,198.630969,231.614397,220.733930,218.446545,228.997807,239.396728,229.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,221.501701,221.202180,220.136943,220.952268,220.733930,220.599636,220.538152,220.924987,220.827205,221.070443,...,284.945417,284.945417,303.076149,273.717803,279.388218,1.000000,274.861279,292.547595,307.186218,308.205143
98069,218.188858,216.728997,217.612557,218.437347,218.446545,218.425849,217.903204,218.294731,218.080630,217.751009,...,284.945417,284.945417,296.803989,284.945417,286.043856,274.861279,1.000000,288.228132,298.909382,295.745653
98159,229.628520,229.010965,229.081126,229.098663,228.997807,229.197283,229.028507,229.181945,228.868383,229.103047,...,284.945417,280.508497,301.000000,291.473751,301.000000,292.547595,288.228132,1.000000,305.138127,306.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [54]:
for i in distances.index[:2]:
    print(distances[i].sort_values()[1:6])

CustomerID
3535    11.488088
264     11.488088
3317    11.488088
2503    11.630146
3305    11.677078
Name: 33, dtype: float64
CustomerID
3317    11.770330
1072    11.908712
3535    12.135529
1920    12.180340
3909    12.357817
Name: 200, dtype: float64


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.